In [1]:
!verdi daemon restart
!reentry scan

Restarting the daemon... OK


In [3]:
import aiida
import os.path as op
from aiida import orm
from aiida.orm import load_code,Dict,StructureData,KpointsData,Int,Str,List,SinglefileData,Float,RemoteData
from aiida.engine import submit
from aiida_pseudo.data.pseudo.psf import PsfData

aiida.load_profile()

# Structure

In [8]:
import sisl
Si64_fdf = sisl.get_sile('/home/aakhtar/Projects/siesta/Obj_my_env/Tests/si64/si64.fdf')
Si64_geom = Si64_fdf.read_geometry()
Si64_geom_ase = Si64_geom.toASE()
Si64_geom_ase.write('host.XSF')
host = StructureData(ase=Si64_geom_ase)
iv=0 
defect = host.clone()
defect.clear_sites()
sites =host.sites
new_sites = sites
vacancy_site = new_sites.pop(iv)
[defect.append_site(s) for s in new_sites]
#len(host.sites)
#len(defect.sites)
ase_defect = defect.get_ase()
ase_defect.write('defect.XSF')

In [ ]:
#from aiida_defects.formation_energy_siesta.formation_energy_siesta_manual_wk_node import FormationEnergyWorkchainSIESTAManual
from aiida_defects.formation_energy_siesta.workflows.formation_energy_siesta_neutral_manual_wk_node import FormationEnergyWorkchainSIESTAManual

In [ ]:
# The pseudopotentials
host_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/home/aakhtar/Calculations/AiiDA/AiiDA-Defects", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        host_pseudos_dict[j]=pseudo

In [ ]:
# The pseudopotentials
defect_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si','Si_ghost'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/home/aakhtar/Calculations/AiiDA/AiiDA-Defects", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        defect_pseudos_dict[j]=pseudo

In [ ]:
# Parameters: very coarse for speed of test
# Note the all the Si atoms are fixed...
parameters = dict={
   "mesh-cutoff": "50 Ry",
   "dm-tolerance": "0.001",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang"
    }
defect_constraints = dict={
    "%block geometryconstraints":
    """
    species-i 2 
    %endblock geometryconstraints"""
    }
relaxation = dict={
    'md-steps': 100
    }
parameters.update(relaxation)
#
# Use this for constraints
#
#parameters.update(constraints)
#
host_parameters = Dict(dict=parameters)
defect_q0_parameters = Dict(dict=parameters)
defect_q_parameters = Dict(dict=parameters)

In [ ]:
host_parameters.get_dict()

In [ ]:
defect_q0_parameters.update_dict(defect_constraints)
defect_q0_parameters.get_dict()

In [ ]:
defect_q_parameters.update_dict(defect_constraints)
defect_q_parameters.get_dict()

In [ ]:
net_charge = dict={'NetCharge':+2.0}
VT = dict={'SaveTotalPotential': True}

In [ ]:
net_charge

In [ ]:
inputs_test = Dict(dict=defect_q_parameters.get_dict())

In [ ]:
inputs_test.update_dict(net_charge)
inputs_test.update_dict(VT)

In [ ]:
inputs_test.get_dict()

In [ ]:
basis = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        %endblock pao-basis-sizes""",})

basis_defect = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        Si_ghost SZ
        %endblock pao-basis-sizes""",})

In [ ]:
#Resources
options = Dict(dict={
    "max_wallclock_seconds": 73600,
    'withmpi': True,
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1,
    }
})


In [ ]:
options.get_dict()

In [ ]:
floating = {'floating_sites': [{"symbols": 'Si',
                                "name": 'Si_ghost',
                               "position": vacancy_site.position}]}

In [ ]:
basis_defect.update_dict(floating)

In [ ]:
basis_defect.get_dict()

In [ ]:
#codename = 'siesta-master@arsalan-pwk'
codename = 'siesta-lua@mylaptop'
code = load_code(codename)

In [ ]:
#The kpoints
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

In [ ]:
inputs = {'host_structure':host,
          'defect_structure':defect,
    'host': {'code':code,
             'parameters':host_parameters,
             'basis':basis,
             'kpoints':kpoints,
             'pseudos':host_pseudos_dict,
             'options':options},
    'defect_q0': {'code':code,
                  'parameters':defect_q0_parameters,
                  'basis':basis_defect,
                  'kpoints':kpoints,
                  'pseudos':defect_pseudos_dict,
                  'options':options},
    'defect_q': {'code':code,
                 'parameters':defect_q_parameters,
                 'basis':basis_defect,
                 'kpoints':kpoints,
                 'pseudos':defect_pseudos_dict,
                 'options':options},
    'defect_charge':Float(-1),
    'defect_site':List(list=[0.0,0.0,0.0]),
    'chemical_potential':Float(-100.629519),
    'epsilon':Float(9.0),
    'wk_node': Int(9282),#Int(9093),
    'add_or_remove':Str('remove'),
    #'correction_scheme':Str('rho')     
}



In [ ]:
inputs

In [ ]:
process = submit(FormationEnergyWorkchainSIESTAManual, **inputs)
print("Submitted VacancyExchangeBarrier workchain; ID={}".format(process.pk))
print(
    "For information about this workchain type: verdi process show {}".format(
        process.pk))
print("For a list of running processes type: verdi process list")


In [ ]:
#node =orm.load_node(9398)

In [ ]:
#node.outputs.defect_q0_structure